In [108]:
import pandas as pd
import numpy as np
from functions import *
from pytorch_util import *
from pytorch_models import ConvBatchLeaky1D,GRU_NCL,LambdaLayer,biasLayer,\
                        MultiHeadedAttention,PositionwiseFeedForward,SublayerConnection
from torch.nn import Linear,Sequential,LeakyReLU,Dropout,L1Loss,BatchNorm1d
from torch.utils.data import DataLoader
from torch.optim import Adam
from functools import partial
from scipy import stats

In [2]:
## change ##
batch_size = 16
clip = .1
block = ResidualBlockGLU
normalFun = normalize_log2
#normalFun = normalize_log
name = 'HandCraft_cnn_shareWeight_attention'

In [3]:
train = pd.read_csv("../Data/train.csv", 
                    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32})
train = train.values

In [211]:
window_length = 256
window_step = 128

def rolling_window(x,window_length,window_step):
    vert_idx_list = np.arange(0, x.size - window_length, window_step)[:,np.newaxis]
    hori_idx_list = np.arange(window_length)
    return x[vert_idx_list + hori_idx_list]

fun_list = [np.min,
            np.max,
            np.std,
            np.mean,
            partial(np.quantile,q=0.25),
            partial(np.quantile,q=0.5),
            partial(np.quantile,q=0.75),
            stats.kurtosis,
            stats.skew
            ]

fun_list = [partial(f,axis=1) for f in fun_list]

def transformFun(x):
    x = rolling_window(x,window_length,window_step)
    x2 = np.stack([f(x) for f in fun_list],1)
    MAE = np.mean(np.abs(x - x2[:,5:6]),1,keepdims=True)
    return np.concatenate([x2,MAE],1)

In [ ]:
# def transformFun(x):
#     x = rolling_window(x[::sample_interval],window_length,window_step)
#     x_diff = np.diff(x)
    
#     x2 = np.stack([f(x) for f in fun_list],1)
#     x2_diff = np.stack([f(x_diff) for f in fun_list],1)
    
#     MAE = np.mean(np.abs(x - x2[:,5:6]),1,keepdims=True)
#     MAE_diff = np.mean(np.abs(x_diff - x2_diff[:,5:6]),1,keepdims=True)
    
#     return np.concatenate([x2,MAE,x2_diff,MAE_diff],1)

Prepare data

In [212]:
train_gen = SequenceGenTransform(train,transformFun)
train_gen = DataLoader(train_gen,batch_size,False,num_workers=2)

val_gen = SequenceGenTransform(train,transformFun,False)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)

Build model

In [135]:
class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadedAttention(2,size,dropout)
        self.feed_forward = PositionwiseFeedForward(size,size,dropout)
        self.sublayer = nn.ModuleList([SublayerConnection(size, dropout),SublayerConnection(size, dropout)])
        self.size = size

    def forward(self, x, mask=None):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)   

In [217]:
model = Sequential(LambdaLayer(lambda x:x.transpose(1,2)),
                   BatchNorm1d(10),
                   #Dropout(0.2),
                   ConvBatchLeaky1D(10,24,5,stride=2),
                   Dropout(0.5),
                   ConvBatchLeaky1D(24,48,5,stride=2),
                   Dropout(0.5),
                   ConvBatchLeaky1D(48,96,5,stride=2),
                   Dropout(0.5),
                   ConvBatchLeaky1D(96,96,5,stride=2),
                   LambdaLayer(lambda x:x.transpose(1,2)),
                   #EncoderLayer(96,0.5),
                   Linear(96,1,bias=False),
                   Dropout(0.2),
                   LambdaLayer(lambda x:x.squeeze(2).mean(1,keepdim=True)),
                   biasLayer((1,)),
                   LambdaLayer(lambda x:x.squeeze(1))
                      ).to('cuda:0')

In [218]:
opt = Adam(trainable_parameter(model),lr=2e-4)

In [219]:
loss_func_nor = loss_func_generator(L1Loss())

Training

In [220]:
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen,clip=clip)

epoch:0, train_loss:5.074368270543905, val_loss:5.4778547286987305
epoch:1, train_loss:4.167447575009787, val_loss:4.23146390914917
epoch:2, train_loss:2.953691764519765, val_loss:3.071362018585205
epoch:3, train_loss:2.2027820514944882, val_loss:2.564425230026245
epoch:4, train_loss:2.0741316120211897, val_loss:2.435102939605713
Training completed in 177.57554006576538s


Submission

In [223]:
submission = pd.read_csv('../Data/sample_submission.csv')

In [224]:
test_gen = SequenceGenTransformTest(submission.seg_id.tolist(),transformFun)
test_gen = DataLoader(test_gen,batch_size,False,num_workers=2)

In [226]:
submission.iloc[:,1] = np.maximum(predict(model,test_gen),0)
submission.to_csv('../Submission/'+name+'.csv',index=False)